In [1]:
import aliases

In [2]:
from everest.ptolemaic.sprites.sprite import Sprite
from everest.utilities import classtools

In [3]:
from everest.ptolemaic.schemas.schema import Schema

Param = Schema.Param

In [4]:
class MySchema(Schema):
    myval: Param.Pos = 1

In [12]:
MySchema.concrete.__slots__

('params', '_softcache', 'weakref')

In [5]:
MySchema(1)

TypeError: cannot create weak reference to 'MySchema_concrete' object

In [6]:
class Tuuple(Sprite):
    reqslots=('foo',)
    content: Sprite.Param.Args
    @classmethod
    def _cls_extra_init_(cls, /):
        defermeths = (
            '__getitem__', '__len__', '__contains__',
            '__iter__', '__reversed__', 'index', 'count',
            )
        for meth in defermeths:
            classtools.add_defer_meth(cls, meth, 'content')

mytup = Tuuple(1, 2, 3)
mytup[0]

1

In [9]:
import weakref

In [10]:
weakref.ref(mytup)

TypeError: cannot create weak reference to 'Tuuple_concrete' object

In [8]:
mytup.__hash__()

8470523098040

In [ ]:
class MyProp(property):

    def __get__(self, instance, owner=None):
        return 'foo'

class MyClass:

    foo = MyProp()

In [ ]:
MyClass

In [ ]:
MyClass().foo

In [ ]:
class MySprite(Sprite):

    a : Sprite.Param.Pos
    b : Sprite.Param.Pos = 'B'
    c : Sprite.Param.PosKw = 'C'
    d : Sprite.Param.PosKw = 'D'
    e : Sprite.Param.Kw
    f : Sprite.Param.Kw = 'F'


class MySubSprite(MySprite):
#     b = 'BB'
    ...

In [ ]:
MySprite('A', e='E')

In [ ]:
MySubSprite('A', e='E')

In [ ]:
MySprite.__signature__

In [ ]:
MySubSprite.__signature__

In [ ]:
from inspect import (
    Parameter as _Parameter,
    _ParameterKind,
    )


class Param:

    __slots__ = ('name', 'default')

    hint = _Parameter.empty
    kind = _ParameterKind.POSITIONAL_OR_KEYWORD

    def __init__(self, name, default=_Parameter.empty, /):
        self.name, self.default = name, default
        self.parameter = _Parameter(
            name, self.kind,
            default=default, annotation=self.hint,
            )

    def __get__(self, instance, /, *, owner=None):
        return getattr(instance.params, self.name)

    @classmethod
    def __class_getitem__(cls, arg: type, /):
        return type(
            f"{cls.__name__}[{arg.__name__}]",
            (cls,),
            dict(hint=arg),
            )

    def _repr(self):
        return f"{self.name}={self.default}"

    def __repr__(self):
        return f"{type(self).__name__}({self._repr()})"


for kind, alias in zip(
        _ParameterKind,
        ('Pos', 'Kw', 'Args', 'Kwargs'),
        ):
    subcls = type(
        f"{Param.__name__}.{alias}",
        (Param,),
        dict(kind=kind),
        )
    setattr(Param, alias, subcls)

In [ ]:
Param.Pos[int]('a', 1)